<a href="https://colab.research.google.com/github/ZohebAbai/DeepLearning-Projects/blob/master/PyTorch_Basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Tensors and Gradients

In [0]:
import torch

A tensor is a number, vector, matrix or any n-dimensional array.

In [2]:
# Number
t1 = torch.tensor(4.)
t1

tensor(4.)

In [3]:
t1.dtype

torch.float32

In [4]:
# Vector
t2 = torch.tensor([1., 2, 3, 4])
t2

tensor([1., 2., 3., 4.])

In [5]:
# Matrix
t3 = torch.tensor([[5.,6], [7,8], [9,10]])
t3

tensor([[ 5.,  6.],
        [ 7.,  8.],
        [ 9., 10.]])

In [6]:
# 3D array
t4 = torch.tensor([
    [[11, 12, 13],
    [13, 14, 15]],
    [[15, 16, 17],
    [17,18,19]]
])
t4

tensor([[[11, 12, 13],
         [13, 14, 15]],

        [[15, 16, 17],
         [17, 18, 19]]])

Tensors can have any number of dimensions, and different lengths along each dimension. 

In [7]:
print(t1.shape)
print(t2.shape)
print(t3.shape)
print(t4.shape)

torch.Size([])
torch.Size([4])
torch.Size([3, 2])
torch.Size([2, 2, 3])


### Tensors operations and gradients

In [8]:
x = torch.tensor(3.)
w = torch.tensor(4., requires_grad=True)
b = torch.tensor(5., requires_grad=True)
y = w*x + b
y

tensor(17., grad_fn=<AddBackward0>)

As expected, y is a tensor with the value 3 * 4 + 5 = 17. What makes PyTorch special is that we can automatically compute the derivative of y w.r.t. the tensors that have requires_grad set to True i.e. w and b. To compute the derivatives, we can call the .backward method on our result y.

In [9]:
# Compute derivatives
y.backward()

#Display the gradients
print('dy/dx:', x.grad)
print('dy/dw:', w.grad)
print('dy/db:', b.grad)

dy/dx: None
dy/dw: tensor(3.)
dy/db: tensor(1.)


### Interoperability with Numpy

In [10]:
import numpy as np
x = np.array([[1,2], [3,4]])
print(x)
y = torch.from_numpy(x)
print(y)

[[1 2]
 [3 4]]
tensor([[1, 2],
        [3, 4]])


In [11]:
print(x.dtype)
print(y.dtype)

int64
torch.int64


In [12]:
z= y.numpy()
print(z)

[[1 2]
 [3 4]]


## Linear Regression and Gradient Descent

In [0]:
# Input (temp, rqinfall, humidity)
inputs = np.array([[73, 67, 43],
                  [91, 88, 64],
                  [87, 134, 58],
                  [102, 43, 37],
                  [69, 96, 70]], dtype= 'float32')

In [0]:
# Targets (apples, oranges)
targets = np.array([[56, 70],
                   [81, 101],
                   [119, 133],
                   [22, 37],
                   [103, 119]], dtype= 'float32')

In [15]:
#Convert inputa and targets to tensors
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


In [16]:
#Weights and biases
w = torch.randn(2,3, requires_grad= True)
b = torch.randn(2, requires_grad= True)
print(w)
print(b)

tensor([[ 0.9525,  0.3977,  0.3852],
        [ 0.6867, -1.0630, -0.1848]], requires_grad=True)
tensor([-0.0781,  0.1031], requires_grad=True)


`torch.randn` creates a tensor with the given shape, with elements picked randomly from a normal distribution with mean 0 and standard deviation 1.

In [0]:
def model(x):
  return x @ w.t() + b 

@ represents matrix multiplication in PyTorch, and the .t method returns the transpose of a tensor.

In [18]:
#Predictions
preds = model(inputs)
print(preds)

tensor([[112.6665, -28.9285],
        [146.2532, -42.7698],
        [158.4252, -93.3038],
        [128.4344,  17.6067],
        [130.7902, -67.4906]], grad_fn=<AddBackward0>)


There’s a huge difference between the predictions of our model, and the actual values of the target variables.

### Loss Function

In [0]:
#MSE loss
def mse(t1, t2):
  diff = t1-t2
  return torch.sum(diff*diff) / diff.numel()

.numel method returns the number of elements in a tensor.

In [20]:
#Compute loss
loss = mse(preds, targets)
print(loss)

tensor(13794.8906, grad_fn=<DivBackward0>)


Lower the loss, better the model

### Compute gradients

In [21]:
loss.backward()
print(w.grad)
print(b.grad)

tensor([[  5255.7065,   4413.2920,   2956.5833],
        [-10967.8496, -13668.3359,  -8070.5430]])
tensor([  59.1139, -134.9772])


### Adjust weights and biases

In [0]:
with torch.no_grad():
  w -= w.grad * 1e-5
  b -= b.grad * 1e-5
  w.grad.zero_()
  b.grad.zero_()

* We use torch.no_grad to indicate to PyTorch that we shouldn’t track, calculate or modify gradients while updating the weights and biases.

* We multiply the gradients with a really small number called the learning rate

*  After we have updated the weights, we reset the gradients to zero by calling .zero_() method. We need to do this, because PyTorch accumulates gradients.

In [23]:
#After
print(w)
print(b)

tensor([[ 0.9000,  0.3536,  0.3557],
        [ 0.7964, -0.9263, -0.1041]], requires_grad=True)
tensor([-0.0787,  0.1045], requires_grad=True)


In [24]:
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(9895.0508, grad_fn=<DivBackward0>)


### Train for multiple epochs

In [28]:
for i in range(100):
  preds = model(inputs)
  loss = mse(preds, targets)
  print('Loss for Epoch', i, ': ', loss )
  loss.backward()
  with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

Loss for Epoch 0 :  tensor(7259.5195, grad_fn=<DivBackward0>)
Loss for Epoch 1 :  tensor(3776.4258, grad_fn=<DivBackward0>)
Loss for Epoch 2 :  tensor(3114.2820, grad_fn=<DivBackward0>)
Loss for Epoch 3 :  tensor(2660.9597, grad_fn=<DivBackward0>)
Loss for Epoch 4 :  tensor(2348.4485, grad_fn=<DivBackward0>)
Loss for Epoch 5 :  tensor(2130.9175, grad_fn=<DivBackward0>)
Loss for Epoch 6 :  tensor(1977.4785, grad_fn=<DivBackward0>)
Loss for Epoch 7 :  tensor(1867.3170, grad_fn=<DivBackward0>)
Loss for Epoch 8 :  tensor(1786.4047, grad_fn=<DivBackward0>)
Loss for Epoch 9 :  tensor(1725.2875, grad_fn=<DivBackward0>)
Loss for Epoch 10 :  tensor(1677.5925, grad_fn=<DivBackward0>)
Loss for Epoch 11 :  tensor(1639.0254, grad_fn=<DivBackward0>)
Loss for Epoch 12 :  tensor(1606.6895, grad_fn=<DivBackward0>)
Loss for Epoch 13 :  tensor(1578.6335, grad_fn=<DivBackward0>)
Loss for Epoch 14 :  tensor(1553.5402, grad_fn=<DivBackward0>)
Loss for Epoch 15 :  tensor(1530.5215, grad_fn=<DivBackward0>)
Lo

In [31]:
print(preds,'\n', targets)

tensor([[ 62.9556,  79.1478],
        [ 83.9398, 103.7895],
        [105.4424, 111.6550],
        [ 54.5994,  87.9964],
        [ 85.3587,  94.8347]], grad_fn=<AddBackward0>) 
 tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


The prediction are now quite close to the target variables, and we can get even better results by training for a few more epochs.